In [2]:
link = "http://web.archive.org/web/20130922081459/http://www.foinse.ie/nuacht/nuacht-is-deanai/6765-suil-go-gcruthofar-158-post-nua-le-tograi-ata-ceadaithe-ag-unag"

In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
page = requests.get(link)
assert page.status_code == 200

In purely text terms, much of the junk can be discarded using these comments:

In [10]:
if "<!-- CONTENT -->" in page.text:
    trim = page.text.split("<!-- CONTENT -->")[1]

In [11]:
if trim and "<!-- //CONTENT -->" in trim:
    trim = trim.split("<!-- //CONTENT -->")[0]

... but it's easier with BeautifulSoup to just extract `<div class="item-page">`

In [14]:
soup = BeautifulSoup(page.text, "lxml")

In [19]:
content = soup.find("div", {"class": "item-page"})

In [22]:
title = content.find("h2").text.strip()

'Súil go gcruthófar 158 post nua le tograí atá ceadaithe ag ÚnaG'

In [28]:
published_tag = content.find("dd", {"class": "published"})

In [29]:
if published_tag:
    published = published_tag.text.strip()

In [31]:
author_tag = content.find("dd", {"class": "createdby"})
if author_tag:
    author = author_tag.text.strip()

In [32]:
author

'Scríofa ag Foinse'

In [38]:
paragraphs_tags = content.find_all("p", {"class": "MsoNormal"})

In [41]:
paragraphs = [p.text.replace("\xa0", " ").strip() for p in paragraphs_tags]

In [42]:
paragraphs

['Ag cruinniú de Bhord Údarás na Gaeltachta sna Forbacha inniu (20 Meán Fómhair), ceadaíodh líon tograí ina mbeifear ag súil go gcruthófar 158 post nua lánaimseartha nuair a bheidh na tograí faoi lán seoil.  Beidh infheistíocht iomlán de €10 milliún i gceist sna tograí seo (óna comhlachtaí agus an tÚdarás) agus tabharfar breis eolais fúthu seo in am trátha.',
 'D’fháiltigh Bord an Údaráis roimh fhógra a rinne Aire Stáit na Gaeltachta níos luaithe an tseachtain seo go bhfuil na Rialacháin sínithe aige chun na critéir pleanála teanga a fhorordú faoi Acht na Gaeltacht, 2012. Is céim eile í seo i dtreo iomlánú an phróisis chun tús a chur le fógairt na Limistéir Pleanála Teanga sa Ghaeltacht, na mBailte Seirbhíse Gaeltachta agus na Líonraí Gaeilge. Tá feidhmeannaigh an Údaráis ag obair i gcomhar leis an Roinn Ealaíon, Oidhreachta agus Gaeltachta agus Foras na Gaeilge leis an chéad chéim eile den phróiseas a ullmhú agus beidh sé seo á fhógairt faoin cheann míosa.',
 'Tugadh le fios fosta ag 

In [46]:
vocab_list = []
for p in paragraphs_tags:
    for vocab in p.find_all("a", {"class": "glossarylink"}):
        item = {}
        item["en"] = vocab.get("title").strip()
        item["ga"] = vocab.text.strip()
        vocab_list.append(item)

In [47]:
vocab_list

[{'en': 'board meeting', 'ga': 'cruinniú de Bhord'},
 {'en': 'amount', 'ga': 'líon'},
 {'en': 'full time', 'ga': 'lánaimseartha'},
 {'en': 'investment', 'ga': 'infheistíocht'},
 {'en': 'criteria', 'ga': 'critéir'},
 {'en': 'In the direction of', 'ga': 'i dtreo'},
 {'en': 'areas', 'ga': 'Limistéir'},
 {'en': 'with/hips', 'ga': 'leis'},
 {'en': 'division/department', 'ga': 'Roinn'},
 {'en': 'the next step', 'ga': 'an chéad chéim eile'},
 {'en': 'it will', 'ga': 'beidh'},
 {'en': 'Also', 'ga': 'fosta'},
 {'en': 'registered', 'ga': 'cláraithe'}]

In [48]:
check = "http://web.archive.org/web/20171222073817/http://www.foinse.ie/nuacht/nuacht-is-deanai/6822-seanoiri-ag-dul-i-mbun-agoide-maidir-le-ciorruithe"

In [49]:
page2 = requests.get(check)
assert page2.status_code == 200

In [50]:
def get_content(url):
    out = {}
    page = requests.get(url)
    if page.status_code != 200:
        return {}

    soup = BeautifulSoup(page.text, "lxml")

    content = soup.find("div", {"class": "item-page"})
    if not content:
        return {}

    title = content.find("h2").text.strip()
    if title:
        out["title"] = title

    published_tag = content.find("dd", {"class": "published"})
    if published_tag:
        out["published"] = published_tag.text.strip()

    author_tag = content.find("dd", {"class": "createdby"})
    if author_tag:
        out["author"] = author_tag.text.strip()

    paragraphs_tags = content.find_all("p", {"class": "MsoNormal"})
    paragraphs = [p.text.replace("\xa0", " ").strip() for p in paragraphs_tags]
    out["text"] = paragraphs

    vocab_list = []
    for p in paragraphs_tags:
        for vocab in p.find_all("a", {"class": "glossarylink"}):
            item = {}
            item["en"] = vocab.get("title").strip()
            item["ga"] = vocab.text.strip()
            vocab_list.append(item)
    out["vocab"] = vocab_list

    return out

In [51]:
get_content(check)

{'author': 'Scríofa ag Úna Ní Eidhin',
 'published': 'Foilsithe an Dé Máirt, 22 Deireadh Fómhair 2013 09:24',
 'text': ['Tá sé i gceist ag seanóirí dul i mbun agóide taobh amuigh den Dáil inniu, mar thoradh ar chiorruithe a fógraíodh sa cháinaisnéis an tseachtain seo caite.',
  'I measc na gciorruithe, bhí laghdú maidir leis an gcárta leighis do dhaoine os cionn 70 bliain d’aois agus rinneadh cinneadh deireadh a chur leis an deontas méala freisin. Tá an grúpa, Parlaimint na Seanóirí in Éirinn, ag rá go bhfuil an-imní ar sheanóirí maidir leis na ciorruithe seo agus go bhfuil feachtas ar bun acu le hiarracht a dhéanamh iallach a chur ar an rialtas, iad a tharraingt siar.',
  'Táthar chun agóid a reáchtáil taobh amuigh de Dáil Éireann ag 1.15pm inniu.',
  'Foinse - News as Gaeilge',
  '',
  'Did you understand this story? Here are the main points:',
  "Senior citizens intend to protest outside the Dáil at 1.15pm today against cuts announced in last week's budget, which included a reductio

In [55]:
def filter_para_list(inlist):
    out = []
    for para in inlist:
        if para == "":
            continue
        elif para.strip() == "Foinse - News as Gaeilge":
            return out
        else:
            out.append(para)
    return out

In [52]:
def extract_summary(inlist):
    if len(inlist) > 2:
        if inlist[-2] == "Did you understand this story? Here are the main points:":
            return inlist[-1]
    return ""